In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [4]:
recommendations_df = pd.read_json('./log_files/ab_test_20250116.log', lines=True)
listening_data = pd.read_json('./data_files/test_sessions.jsonl', lines=True)

In [5]:
results = []
# WINDOW_DAYS = 7

In [8]:
for index, recommendation in recommendations_df.iterrows():
    rec_time = pd.to_datetime(recommendation['timestamp'])
    # end_time = rec_time + timedelta(days=WINDOW_DAYS)
    
    group_users = recommendation['user_ids']
    recommended_tracks = recommendation['recommended_tracks']
    
    group_listening = listening_data[
        (listening_data['user_id'].isin(group_users)) #&
       # (listening_data['timestamp'] >= rec_time) &
       # (listening_data['timestamp'] <= end_time)
    ]
    
    track_stats = {}
    for track in recommended_tracks:
        listeners = group_listening[group_listening['track_id'] == track]['user_id'].nunique()
        track_stats[track] = listeners
    
    result = {
        'recommendation_id': index,
        'timestamp': rec_time,
        'model_type': recommendation['model_type'],
        'group_size': len(group_users),
        
        'avg_listeners': np.mean(list(track_stats.values())),
        
        'tracks_with_listeners': sum(1 for v in track_stats.values() if v > 0),
        
        'tracks_listened_by_half': sum(1 for v in track_stats.values() if v >= len(group_users)/2),
        
        'tracks_listened_by_all': sum(1 for v in track_stats.values() if v == len(group_users))
    }
    
    for user in group_users:
        user_listening = group_listening[group_listening['user_id'] == user]
        listened_tracks = user_listening['track_id'].unique()
        user_listened_count = sum(1 for track in recommended_tracks if track in listened_tracks)
        result[f'user_{user}_listened'] = user_listened_count
    
    results.append(result)

In [9]:
analysis_df = pd.DataFrame(results)

In [17]:
print("\Average statistics model wise:")
display(analysis_df.groupby('model_type')[['avg_listeners', 'tracks_with_listeners', 'tracks_listened_by_half', 'tracks_listened_by_all']].mean())

\Average statistics model wise:


,avg_listeners,tracks_with_listeners,tracks_listened_by_half,tracks_listened_by_all
model_type,,,,
advanced,0.200000,5.0,0.0,0.0
basic,0.366667,8.0,0.0,0.0
